In [9]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import xgboost as xgb
import datetime
import matplotlib.pyplot as plt

In [10]:
# csv 파일 불러오기
race_result = pd.read_csv(r'C:\Users\po031\Desktop\hyunse_df.csv', encoding = 'utf-8')

In [11]:
# csv 파일 column 보기
race_result.columns

Index(['Unnamed: 0', 'url', 'date', 'round', 'name', 'code', 'rank', 'lane',
       'sex', 'age', 'jockey_w', 'rating', 'jockey', 'difference', 'weight',
       'dandivi', 'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr',
       's1f', 'c1', 'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m',
       'weight_diff', 'raw_weight', 'weight_added', 'distance', 'velocity',
       'prev1_rank', 'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo',
       'prev3_velo', 'horse_level_num', 'year', 'month', 'choice', 'choice2',
       'choice3', 'danwin1yr', 'bokwin1yr', 'samwin1yr', 'jdanwin1yr',
       'jbokwin1yr', 'jsamwin1yr', 'jprev1_rank', 'jprev2_rank', 'jprev3_rank',
       'humidity', 'race_level', 'new_distance', 'groupid', 'avg_vel',
       'adj_vel', 'prev_rank', 'false1time', 'false2time', 'jprev_rank',
       'jfalse1time', 'jfalse2time', 'norating', 'sex1', 'sex2'],
      dtype='object')

In [12]:
# sex를 numeric으로 바꾸기
def sex_num(x):
    if x == '암': return 0
    elif x == '수': return 1
    else: return 2

In [13]:
# 변수를 조합하여 새로운 변수 생성
    race_result['sex'] = race_result['sex'].apply(sex_num)

In [14]:
# XGBoost에 넣을 변수만 추출하여 df 생성
li = ['record', 'round', 'lane', 'sex', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'weight_diff', 'raw_weight',
      'distance', 'horse_level_num', 'year', 'month', 'new_distance', 'avg_vel', 'adj_vel', 'prev_rank', 'jprev_rank',
      'norating']
df = race_result[li].dropna()
df

,record,round,lane,sex,age,jockey_w,dandivi,yeondivi,cure_in_1m,weight_diff,...,distance,horse_level_num,year,month,new_distance,avg_vel,adj_vel,prev_rank,jprev_rank,norating
0,111.2,10,10,1,4,54.0,1.9,1.2,0,-7.0,...,1700.0,3.0,2018,10,1,14.950846,1.022535,4.0,5.5,0
1,111.5,10,3,2,6,54.5,40.2,8.1,0,-1.0,...,1700.0,3.0,2018,10,1,14.950846,1.019784,3.5,7.5,0
2,111.7,10,11,0,3,55.5,4.7,2.0,1,-5.0,...,1700.0,3.0,2018,10,0,14.950846,1.017958,3.0,6.5,0
3,111.7,10,6,2,3,52.0,14.5,3.6,1,7.0,...,1700.0,3.0,2018,10,0,14.950846,1.017958,2.5,6.5,0
4,112.0,10,2,2,3,52.0,13.8,3.6,1,-1.0,...,1700.0,3.0,2018,10,1,14.950846,1.015232,4.5,6.5,0
5,112.4,10,4,1,3,52.0,7.5,2.3,0,-13.0,...,1700.0,3.0,2018,10,0,14.950846,1.011619,4.5,7.5,0
6,113.0,10,12,1,5,52.5,27.8,4.2,0,-21.0,...,1700.0,3.0,2018,10,1,14.950846,1.006247,4.0,5.0,0
7,113.2,10,1,2,3,56.0,10.9,2.5,1,0.0,...,1700.0,3.0,2018,10,0,14.950846,1.004469,6.5,6.5,0
8,113.3,10,9,2,5,53.5,22.3,3.4,0,-1.0,...,1700.0,3.0,2018,10,1,14.950846,1.003583,4.5,5.5,0
9,114.1,10,5,2,3,52.0,69.5,11.8,0,8.0,...,1700.0,3.0,2018,10,1,14.950846,0.996546,8.0,5.5,0


In [16]:
# train-test split with cross validation
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    train = df.iloc[idx_train]
    test = df.iloc[idx_test]
    
    # train, test dataframe을 array로 바꾸기
    train_array = train.as_matrix()
    test_array = test.as_matrix()
    
    # 맨 앞 칼럼이 record time(종속변수)이라서 따로 떼어줌.
    real1 = train_array[:,0]; real2 = test_array[:,0]
    
    # xgb에서 쓸 수 있도록 자료형 변경
    xgb_train = xgb.DMatrix(train_array[:,1:], label=real1) 
    xgb_test = xgb.DMatrix(test_array[:,1:], label=real2)
   
#     # XGBoost sample1 만들기
#     xgb1 = xgb.XGBClassifier(eta=0.01,
#                              n_estimators=1000, 
#                              max_depth=20,
#                              min_child_weight=1,
#                              gamma=0,
#                              subsample=0.8,
#                              colsample_bytree=0.8,
#                              objective= 'reg:linear',
#                              scale_pos_weight=1,
#                              seed=27)
#     xgb1_param = xgb1.get_xgb_params()
    
#     # train & test
#     num_round = 200
#     bst = xgb.train(xgb1_param, xgb_train, num_round) # train
#     pred1 = bst.predict(xgb_train) # test
#     pred2 = bst.predict(xgb_test)
    
#     # Mean Absolute Error 출력
#     print('order: {0}, train_error: {1}, test_error: {2}'.format(i, abs(pred1 - real1).mean(), abs(pred2 - real2).mean()))
    
    # XGBoost sampe2(max_depth 구하기) 만들기
    for j in range(10,20,2):
        xgb2 = xgb.XGBClassifier(learning_rate=0.1,
                                 n_estimators=1000, 
                                 max_depth=j,
                                 min_child_weight=1,
                                 gamma=0,
                                 subsample=0.8,
                                 colsample_bytree=0.8,
                                 objective= 'reg:linear',
                                 scale_pos_weight=1,
                                 seed=27)
        xgb2_param = xgb2.get_xgb_params()
        # train & test
        num_round = 200
        bst = xgb.train(xgb2_param, xgb_train, num_round) # train
        pred1 = bst.predict(xgb_train) # test
        pred2 = bst.predict(xgb_test)
        
        # Mean Absolute Error 출력
        print('max_depth: {0}, train_error: {1}, test_error: {2}'.format(j, abs(pred1-real1).mean(), abs(pred2-real2).mean()))

max_depth: 10, train_error: 0.6553835604142937, test_error: 1.9441082123722486
max_depth: 12, train_error: 0.263460896831064, test_error: 1.8909369800060294
max_depth: 14, train_error: 0.0927803719550044, test_error: 1.9126643934662628
max_depth: 16, train_error: 0.02064905107090526, test_error: 1.891241479542453


KeyboardInterrupt: 

In [ ]:
# train-test split with cross validation
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    train = df.iloc[idx_train]
    test = df.iloc[idx_test]
    
    # train, test dataframe을 array로 바꾸기
    train_array = train.as_matrix()
    test_array = test.as_matrix()
    
    # 맨 앞 칼럼이 record time(종속변수)이라서 따로 떼어줌.
    real1 = train_array[:,0]; real2 = test_array[:,0]
    
    # xgb에서 쓸 수 있도록 자료형 변경
    xgb_train = xgb.DMatrix(train_array[:,1:], label=real1) 
    xgb_test = xgb.DMatrix(test_array[:,1:], label=real2)
   
#     # XGBoost sample1 만들기
#     xgb1 = xgb.XGBClassifier(eta=0.01,
#                              n_estimators=1000, 
#                              max_depth=20,
#                              min_child_weight=1,
#                              gamma=0,
#                              subsample=0.8,
#                              colsample_bytree=0.8,
#                              objective= 'reg:linear',
#                              scale_pos_weight=1,
#                              seed=27)
#     xgb1_param = xgb1.get_xgb_params()
    
#     # train & test
#     num_round = 200
#     bst = xgb.train(xgb1_param, xgb_train, num_round) # train
#     pred1 = bst.predict(xgb_train) # test
#     pred2 = bst.predict(xgb_test)
    
#     # Mean Absolute Error 출력
#     print('order: {0}, train_error: {1}, test_error: {2}'.format(i, abs(pred1 - real1).mean(), abs(pred2 - real2).mean()))